# Contexte 
Le Problème du Voyageur de Commerce (TSP) pose la question suivante : « Étant donnée une liste de
villes et les distances entre chaque paire de villes, quel est le plus court trajet possible qui visite chaque
ville et revient à la ville d’origine ? » Il s’agit d’un problème N P-difficile en optimisation combinatoire,
important en recherche opérationnelle et en informatique théorique.
Le problème a été formulé pour la première fois en 1930 et est l’un des problèmes les plus étudiés
en optimisation. Il est utilisé comme benchmark pour de nombreuses méthodes d’optimisation. Bien que
le problème soit computationnellement difficile, de nombreuses heuristiques et algorithmes exacts sont
connus, de sorte que certaines instances avec des dizaines de milliers de villes peuvent être résolues com-
plètement et même des problèmes avec des millions de villes peuvent être approximés à une fraction de
1%.
Formellement, soit G = (V,E,w) un graphe complet non orienté tel que |V| = n et w : E → R≥0
la fonction de poids. C’est-à-dire que G est le graphe Kn avec des poids sur ses arêtes. Dans ce modèle,
chaque sommet représente une ville et chaque arête pondérée reliant deux sommets représente la distance
entre les villes correspondantes.
Ainsi, étant donné un graphe complet pondéré non orienté, le TSP consiste à trouver un cycle ha-
miltonien (un cycle qui traverse chaque sommet exactement une fois) qui minimise la somme de ses
poids.

# Fonctions

In [5]:
def get_data_1(path : str) -> list:
    """
    get_data_1("Projet/17.in")
    """
    D = []
    with open(path, "r") as f:
        N  = int(f.readline())

        for _ in range(N):
            D.append(list(map(int, f.readline().split())))
    return D

In [6]:
import math

def read_tsp_file(filename):
    """
    Lit un fichier au format TSPLIB (.tsp) avec des coordonnées géographiques.
    
    Structure du fichier :
    - En-tête avec métadonnées (NAME, TYPE, DIMENSION, EDGE_WEIGHT_TYPE, etc.)
    - Section NODE_COORD_SECTION avec les coordonnées de chaque ville
    - EOF pour marquer la fin
    
    Args:
        filename (str): Le chemin vers le fichier .tsp
        
    Returns:
        n (int): Nombre de villes
        coords (list of tuples): Liste des coordonnées [(lat1, lon1), (lat2, lon2), ...]
        edge_weight_type (str): Type de distance (GEO, EUC_2D, etc.)
    """
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    # Variables pour stocker les informations
    n = 0
    edge_weight_type = None
    coords = []
    in_coord_section = False
    
    for line in lines:
        line = line.strip()
        
        # Parse les métadonnées de l'en-tête
        if line.startswith("DIMENSION"):
            # Format: "DIMENSION: 535" ou "DIMENSION : 535"
            n = int(line.split(":")[-1].strip())
        
        elif line.startswith("EDGE_WEIGHT_TYPE"):
            # Format: "EDGE_WEIGHT_TYPE: GEO"
            edge_weight_type = line.split(":")[-1].strip()
        
        # Détection du début de la section des coordonnées
        elif line == "NODE_COORD_SECTION":
            in_coord_section = True
            continue
        
        # Détection de la fin du fichier
        elif line == "EOF":
            break
        
        # Lecture des coordonnées
        elif in_coord_section and line:
            parts = line.split()
            if len(parts) >= 3:
                # Format: "1  36.49  7.49"
                # On ignore l'index (parts[0]) et on prend lat/lon
                node_id = int(parts[0])
                lat = float(parts[1])
                lon = float(parts[2])
                coords.append((lat, lon))
    
    return n, coords, edge_weight_type

def calculate_geo_distance(coord1, coord2):
    """
    Calcule la distance géographique entre deux points (format TSPLIB GEO).
    
    Formule utilisée par TSPLIB pour EDGE_WEIGHT_TYPE: GEO
    (approximation sphérique de la Terre)
    
    Args:
        coord1 (tuple): (latitude, longitude) du point 1
        coord2 (tuple): (latitude, longitude) du point 2
        
    Returns:
        int: Distance arrondie en kilomètres
    """
    PI = 3.141592
    RRR = 6378.388  # Rayon de la Terre en km (valeur TSPLIB)
    
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    # Conversion des degrés décimaux en radians (formule TSPLIB)
    deg1 = int(lat1)
    min1 = lat1 - deg1
    lat1_rad = PI * (deg1 + 5.0 * min1 / 3.0) / 180.0
    
    deg2 = int(lon1)
    min2 = lon1 - deg2
    lon1_rad = PI * (deg2 + 5.0 * min2 / 3.0) / 180.0
    
    deg1 = int(lat2)
    min1 = lat2 - deg1
    lat2_rad = PI * (deg1 + 5.0 * min1 / 3.0) / 180.0
    
    deg2 = int(lon2)
    min2 = lon2 - deg2
    lon2_rad = PI * (deg2 + 5.0 * min2 / 3.0) / 180.0
    
    # Formule de distance sphérique
    q1 = math.cos(lon1_rad - lon2_rad)
    q2 = math.cos(lat1_rad - lat2_rad)
    q3 = math.cos(lat1_rad + lat2_rad)
    
    distance = RRR * math.acos(0.5 * ((1.0 + q1) * q2 - (1.0 - q1) * q3)) + 1.0
    
    return int(distance)

def calculate_euclidean_distance(coord1, coord2):
    """
    Calcule la distance euclidienne 2D entre deux points.
    
    Args:
        coord1 (tuple): (x, y) du point 1
        coord2 (tuple): (x, y) du point 2
        
    Returns:
        int: Distance euclidienne arrondie
    """
    x1, y1 = coord1
    x2, y2 = coord2
    
    distance = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    
    return int(distance + 0.5)  # Arrondi standard TSPLIB

def build_distance_matrix(coords, edge_weight_type):
    """
    Construit la matrice de distances à partir des coordonnées.
    
    Args:
        coords (list): Liste des coordonnées
        edge_weight_type (str): Type de calcul ("GEO" ou "EUC_2D")
        
    Returns:
        matrix (list of list): Matrice de distances nxn
    """
    n = len(coords)
    matrix = [[0] * n for _ in range(n)]
    
    # Choisir la fonction de distance appropriée
    if edge_weight_type == "GEO":
        distance_func = calculate_geo_distance
    else:  # Par défaut, Euclidienne
        distance_func = calculate_euclidean_distance
    
    # Remplir la matrice (symétrique)
    for i in range(n):
        for j in range(i + 1, n):
            dist = distance_func(coords[i], coords[j])
            matrix[i][j] = dist
            matrix[j][i] = dist  # Symétrie
    
    return matrix


# filename = "donnees_autre/ali535.tsp"  
# n, coords, edge_weight_type = read_tsp_file(filename)
# matrix = build_distance_matrix(coords, edge_weight_type)

# Question 1
Décrire (d’autres) situations réelles qui peuvent être modélisées comme un TSP.
- Un ensemble de mots est donné. Le but est de passer par chaque mot en éffectuant le moins d'action possible. Chaque mot peut se transformer en un autre en :
    - changeant une lettre
    - ajoutant une lettre
    - enlevant une lettre 
- Un telescope doit observer un certain nombre de planetes. Le changement de possition de la lunette est couteux. Le but est de prévoir le chemin le plus court passant par toutes les planètes, afin de réduire les coûts.
- Un personnage sur Amongus doit effectuer toutes ses tâches. Etant donné la présence d'imposteurs dans le groupe, il doit effectuer ses tâches le plus vite possible

# Question 2
Développer et implémenter un algorithme exact pour résoudre le TSP, utilisant la technique du
Branch and Bound.

In [ ]:
import pulp
 
def solve_tsp_optimized(distances):

    n = len(distances)

    prob = pulp.LpProblem("TSP_Iterative", pulp.LpMinimize)
 
    x = pulp.LpVariable.dicts("x", 
                             ((i, j) for i in range(n) for j in range(n) if i != j), 
                             cat='Binary')
 
    prob += pulp.lpSum(distances[i][j] * x[i, j] for i in range(n) for j in range(n) if i != j)
 
    for i in range(n):

        prob += pulp.lpSum(x[i, j] for j in range(n) if i != j) == 1
        prob += pulp.lpSum(x[j, i] for j in range(n) if i != j) == 1

    iteration = 0

    while True:

        prob.solve(pulp.PULP_CBC_CMD(msg=0))

        if prob.status != pulp.LpStatusOptimal:

            print("Problem is infeasible.")

            return None, None
 
        current_x = {(i, j): pulp.value(x[i, j]) for i in range(n) for j in range(n) if i != j}

        subtours = find_subtours(current_x, n)
 
        print(f"Iteration {iteration}: Found {len(subtours)} subtours. Cost: {pulp.value(prob.objective)}")
 
        if len(subtours) == 1:
            best_sol = current_x
            break
 
        for S in subtours:
            prob += pulp.lpSum(x[i, j] for i in S for j in S if i != j) <= len(S) - 1

        iteration += 1
 
    return pulp.value(prob.objective), extract_tour(best_sol, n)
  
def find_subtours(x_values, n):

    """
    Finds connected components (subtours) in the current solution.
    Since variables are binary, we just follow edges where x[i,j] == 1.
    """

    visited = [False] * n
    subtours = []
    adj = {i: [] for i in range(n)}

    for (i, j), val in x_values.items():
        if val > 0.9:
            adj[i].append(j)
 
    for i in range(n):
        if not visited[i]:
            component = []
            stack = [i]
            visited[i] = True

            while stack:
                u = stack.pop()
                component.append(u)
                for v in adj[u]:
                    if not visited[v]:
                        visited[v] = True
                        stack.append(v)
            subtours.append(component)
    return subtours
 
def extract_tour(x_values, n):

    """Reconstructs the path from the solution dictionary"""

    start = 0
    tour = [start]
    current = start
    visited_count = 0
    while visited_count < n - 1:
        for j in range(n):
            if current != j and x_values.get((current, j), 0) > 0.9:
                tour.append(j)
                current = j
                visited_count += 1
                break

    return tour

In [ ]:
dist_matrix = get_data_1("Projet/100.in")
n = len(dist_matrix)

cout, chemin = solve_tsp_optimized(dist_matrix)

print("-" * 30) 
print(f"Coût Optimal : {cout}")
print(f"Chemin (Tour) : {chemin}")

--- Démarrage de la résolution TSP ---
Iteration 0: Found 45 subtours. Cost: 17087.0
Iteration 1: Found 26 subtours. Cost: 19604.0
Iteration 2: Found 8 subtours. Cost: 20793.0
Iteration 3: Found 7 subtours. Cost: 20871.0
Iteration 4: Found 8 subtours. Cost: 21096.0
Iteration 5: Found 6 subtours. Cost: 21187.0
Iteration 6: Found 3 subtours. Cost: 21222.0
Iteration 7: Found 3 subtours. Cost: 21267.0
Iteration 8: Found 2 subtours. Cost: 21268.0
Iteration 9: Found 1 subtours. Cost: 21282.0
------------------------------
Coût Optimal : 21282.0
Chemin (Tour) : [0, 46, 92, 27, 66, 57, 60, 50, 86, 24, 80, 68, 63, 39, 53, 1, 43, 49, 72, 67, 84, 81, 94, 12, 75, 32, 36, 4, 51, 77, 95, 38, 29, 47, 99, 40, 70, 13, 2, 42, 45, 28, 33, 82, 54, 6, 8, 56, 19, 11, 26, 85, 34, 61, 59, 76, 22, 97, 90, 44, 31, 10, 14, 16, 58, 73, 20, 71, 9, 83, 35, 98, 37, 23, 17, 78, 52, 87, 15, 93, 21, 69, 65, 25, 64, 3, 96, 55, 79, 30, 88, 41, 7, 91, 74, 18, 89, 48, 5, 62]


In [ ]:
filename = "donnees_autre/ali535.tsp"  
n, coords, edge_weight_type = read_tsp_file(filename)
matrix = build_distance_matrix(coords, edge_weight_type)
cout, chemin = solve_tsp_optimized(matrix)

print("-" * 30) 
print(f"Coût Optimal  : {cout}")
print(f"Chemin (Tour) : {chemin}")

Iteration 0: Found 247 subtours. Cost: 156186.0
Iteration 1: Found 117 subtours. Cost: 190150.0
Iteration 2: Found 58 subtours. Cost: 198672.0
Iteration 3: Found 25 subtours. Cost: 200080.0


# QUESTION 3
Développer et implémenter une heuristique constructive pour résoudre le TSP.